We begin Phase III through the replication of the winning solution by Chris Detotte and Konstatin Yalenkov. The first step will be a more mature EDA, we will first follow alijs' notebook as Chris uses it as the basis of their analysis, and then replicate the innovations that Chris makes.

Link to alijs' EDA notebook: https://www.kaggle.com/alijs1/ieee-transaction-columns-reference

Link to Chris' EDA notebook: https://www.kaggle.com/cdeotte/eda-for-columns-v-and-id

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import warnings
warnings.filterwarnings("ignore")
cols_to_show = 150

In [34]:
def h(content):
    display(HTML(content))
    
def timehist1(col):
    #Not sure what this is doing???
    N = 8000 if col in ['TransactionAmt'] else 99999999999999
    
    tr[tr['isFraud']==0].set_index('TransactionDT')[col].clip(0, N).plot(style='.', title='Hist ' + col, figsize=(15,3))
    tr[tr['isFraud']==1].set_index('TranasactionDT')[col].clip(0,N).plot(style='.', title='Hist ' + col, figsize=(15,3))
    te.set_index('TransactionDT')[col].clip(0,N).plot(style='.', title='Blue = no fraud, orange = fraud, green = test', figsize=(15,3))
    plt.show()
    
def _desc(data, col, label):
    d0 = data.describe().reset_index()
    d0.columns = [col, label]
    return d0.append({col:'unique values', label:data.unique().shape[0]}, ignore_index=True) \
            .append({col:'NaNs', label:data.isnull().sum()}, ignore_index=True) \
            .append({col:'NaN share', label:np.round(data.isnull().sum()/data.shape[0], 4)}, ignore_index=True)

def desc1(col):
    d0 = _desc(tr[col], col, 'Train')
    d1 = _desc(tr.loc[tr['isFraud']==1, col], col, 'Train fraud')
    d2 = _desc(tr.loc[tr['isFraud']==0, col], col, 'Train not fraud')
    d3 = _desc(te[col], col, 'Test')
    n_public = int(te.shape[0]*0.2)
    d4 = _desc(te.loc[:n_public, col], col, 'Test public')
    d5 = _desc(te.loc[n_public:, col], col, 'Test private')
    dd = d0.merge(d1).merge(d2).merge(d3).merge(d4).merge(d5)
    display(dd)
    
    if col not in ['TransactionID']:
        h('<b>Most popular values (NaN = -999): </b>')
        N = 10
        d0 = tr[['isFraud',col]].fillna(-999).groupby(col)['isFraud'].agg(['size','mean','sum']).reset_index().sort_values('size',ascending=False).reset_index(drop=True)
        


In [5]:
tr = pd.read_csv('Data/train_transaction.csv')
te = pd.read_csv('Data/test_transaction.csv')

In [35]:
col = 'TransactionAmt'
desc1(col)

,TransactionAmt,Train,Train fraud,Train not fraud,Test,Test public,Test private
0,count,590540.000000,20663.000000,569877.000000,506691.000000,101339.000000,405353.000000
1,mean,135.027176,149.244779,134.511665,134.725568,141.732271,132.973690
2,std,239.162522,232.212163,239.395078,245.779822,257.473804,242.736757
3,min,0.251000,0.292000,0.251000,0.018000,0.018000,0.140000
4,25%,43.321000,35.044000,43.970000,40.000000,45.950000,39.000000
5,50%,68.769000,75.000000,68.500000,67.950000,72.950000,67.950000
6,75%,125.000000,161.000000,120.000000,125.000000,134.950000,120.950000
7,max,31937.391000,5191.000000,31937.391000,10270.000000,9154.460000,10270.000000
8,unique values,20902.000000,2515.000000,20560.000000,14119.000000,5144.000000,12117.000000
9,NaNs,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
tr.groupby('P_emaildomain')